# ExplainableRL: Onboarding recipe
In this notebook, a template sequence of steps to use the ExplainableRL library is described.

In [1]:
# TODO: give a bit of context and overview

### Relevant imports

In [ ]:
from library import *

# Import functions
from src.foundation.engine import Engine
from src.data_handler.data_handler import DataHandler
from src.explainability.pdp import PDP
from src.explainability.shap_values import ShapValues

### 1. Parameters Setting

In [2]:
# TODO: Explain each parameter and when to use what

### 2. Initialisation

In [4]:
# TODO: overview of what to initialise (DataHandler & Engine with Agent and Env inside)
# TODO: If needed by evaluation/explainability/performance add a new dictionary and explain it
hyperparam_dict = {}
verbose = hyperparam_dict['verbose']

#### 2.a. Data Loading

In [ ]:
# TODO: Explain data strucuture needed for both Env & how DataHanlder is built

#### 2.b. Engine Initialisation

In [ ]:
# TODO: Load and split the data in train and test and save the test set
    def load_data(self, delimiter=','):
        """Load data from file.

        Args:
            delimiter (str): Which separates columns.
        """
        file_type = self.data_path.split('.')[-1]
        if file_type == 'csv':
            self.dataset = pd.read_csv(self.data_path, sep=delimiter)
        elif file_type == 'xlsx':
            self.dataset = pd.read_excel(self.data_path)
        elif file_type == 'parquet':
            self.dataset = pd.read_parquet(self.data_path)
        else:
            raise ValueError("File type not supported")

In [ ]:
# TODO: Add explanation
if verbose:
    timestamp = datetime.now().strftime("%d/%m/%Y %H:%M:%S")
    print(f"{timestamp}: Load data")
dh = DataHandler(hyperparam_dict=hyperparam_dict)

#### 2.c. Environment and Agent Initialisation

In [ ]:
# TODO: Add explanation
if verbose:
    timestamp = datetime.now().strftime("%d/%m/%Y %H:%M:%S")
    print(f"{timestamp}: Initialise Engine")
engine = Engine(dh=dh, hyperparam_dict=hyperparam_dict)


In [ ]:
if verbose:
    timestamp = datetime.now().strftime("%d/%m/%Y %H:%M:%S")
    print(f"{timestamp}: Create the world")
engine.create_world()

### 3. Agent Training and Saving

In [ ]:
# TODO: Add explanation

#### 3.a. Training

In [ ]:
# TODO: Add explanation
if verbose:
    timestamp = datetime.now().strftime("%d/%m/%Y %H:%M:%S")
    n_samples = hyperparam_dict['n_samples']
    n_episodes = hyperparam_dict['n_episodes']
    print(f"{timestamp}: Train the agent on {n_samples} samples and {n_episodes} episodes")
engine.train_agent()

#### 3.b. Saving

In [ ]:
# TODO: Add explanation
# TODO: Add saving and uploading process in pickle
# TODO: Take path and file name from dictionary

### 4. Agent Evaluation

In [ ]:
# TODO: Add explanation
# TODO: Adapt to new main - Change and adapt

if verbose:
    timestamp = datetime.now().strftime("%d/%m/%Y %H:%M:%S")
    print(f"{timestamp}: Evaluate agent")

states, actions, b_actions, rewards_hist, actions_agent, b_actions_agent, rewards_agent = \
    engine.evaluate_agent()

# Sum obtained reward optimal vs historical policy
print(f"Return based on historical data: {np.sum(rewards_hist)}")
print(f"Return based on agent policy: {np.sum(rewards_agent)}")

# Plot perfomance
plt.scatter(actions, actions_agent)
plt.savefig('policy.png')

### 5. Explainability

In [ ]:
# TODO: Add explanation on explainability framework

#### 5.a. PDP Plots

In [ ]:

# TODO: Add explanation
if verbose:
    timestamp = datetime.now().strftime("%d/%m/%Y %H:%M:%S")
    print(f"{timestamp}: Show PDPs plots")
pdp = PDP(engine=engine)
pdp.build_data_for_plots(engine.agent.Q, engine.agent.Q_num_samples)
type_features = hyperparam_dict['feature_types']
fig_name = "PDP plots - All states"
pdp.plot_pdp(states_names=state_labels, 
             fig_name=fig_name,
             type_features=type_features, 
             savefig=True, 
             all_states=True)
# TODO: Change PDP parameters based on new parameters dictionary

#### 5.b. Shap Values

In [ ]:
# TODO: Add explanation
if verbose:
    timestamp = datetime.now().strftime("%d/%m/%Y %H:%M:%S")
    print(f"{timestamp}: Show SHAP values plots")
shap_values = ShapValues(sample=[8, 1, 1, 1],
                         engine=engine,
                         number_of_samples=shap_num_samples)
shaps, predicted_action = shap_values.compute_shap_values()
print(shaps)
print(predicted_action)
# TODO: Change SHAP parameters based on new parameters dictionary

### 6. Performance

In [ ]:
# TODO: Add explanation
# Explain how to run the performance and what all the results mean

### 7. Documentation

In [ ]:
# TODO: Add explanation
# Explain how to access documentation in the browser